In [23]:
#Siamese Architecture_Simple_Bi_LSTM
#keras_1.1.2_theano_backend
#Dropout=0.5,0.5
#epoch=10
#train:test:validation=8:2
#Using Glove for words

#parameters for the Model
#MAX_SEQUENCE_LENGTH = 200
#MAX_NB_WORDS = 40000
#EMBEDDING_DIM = 100
#VALIDATION_SPLIT = 0.2


#Result:
#Stopped at 8,as no improvement

In [2]:
import os
with open(os.path.expanduser('~')+'/.keras/keras.json','w') as f:
    new_settings = """{\r\n
    "epsilon": 1e-07,\r\n
    "image_data_format": "channels_last",\n
    "backend": "theano",\r\n
    "floatx": "float32"\r\n
    }"""
    f.write(new_settings)

import keras

Using Theano backend.


In [3]:
import pandas as pd
import nltk
import numpy as np
import gensim

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout, LSTM, GRU, Bidirectional
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializations
from keras.callbacks import ModelCheckpoint

from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
import numpy.random as rng
import numpy as np
import os
import dill as pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [5]:
train_DF=pd.read_csv('train.csv',index_col='id')
train_DF.reset_index(drop=True,inplace=True)
train_DF['question1'] = train_DF['question1'].astype(str)
train_DF['question2'] = train_DF['question2'].astype(str)

In [6]:
#parameters for the Model
MAX_SEQUENCE_LENGTH = 200
MAX_NB_WORDS = 40000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [7]:
import re
numbers=re.compile(r'(\d+\.\d+|\d+)')
def cleanText(text):
    return numbers.sub('N',text)

In [8]:
texts = []
questions1=[]
questions2=[]
duplicates=[]

for idx in range(train_DF.question1.shape[0]):
    text=train_DF.question1[idx]
    text=cleanText(text)
    texts.append(text)
    questions1.append(text)
    
    text=train_DF.question2[idx]
    text=cleanText(text)
    texts.append(text)
    questions2.append(text)
    
    duplicates.append(train_DF.is_duplicate[idx])

In [9]:
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

Total 90278 unique tokens.


In [10]:
questions1_sequences = tokenizer.texts_to_sequences(questions1)
questions2_sequences = tokenizer.texts_to_sequences(questions2)

In [11]:
questions1_sequences = pad_sequences(questions1_sequences, maxlen=MAX_SEQUENCE_LENGTH)
questions2_sequences = pad_sequences(questions2_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [12]:
questions2_sequences.shape

(404290, 200)

In [13]:
duplicates_1=duplicates
duplicates = to_categorical(np.asarray(duplicates))
print('Shape of label tensor:', duplicates.shape)

Shape of label tensor: (404290, 2)


In [14]:
indices = np.arange(questions1_sequences.shape[0])
np.random.shuffle(indices)

questions1_sequences = questions1_sequences[indices]
questions2_sequences = questions2_sequences[indices]
duplicates = duplicates[indices]

nb_validation_samples = int(VALIDATION_SPLIT * questions1_sequences.shape[0])

questions1_sequences_train = questions1_sequences[:-nb_validation_samples]
questions2_sequences_train = questions2_sequences[:-nb_validation_samples]
duplicates_train = duplicates[:-nb_validation_samples]

questions1_sequences_val = questions1_sequences[-nb_validation_samples:]
questions2_sequences_val = questions2_sequences[-nb_validation_samples:]
duplicates_val = duplicates[-nb_validation_samples:]

In [15]:
print('Traing and validation set number of positive and negative reviews')
print(duplicates_train.sum(axis=0))
print(duplicates_val.sum(axis=0))

Traing and validation set number of positive and negative reviews
[ 203917.  119515.]
[ 51110.  29748.]


In [16]:
embeddings_index = {}
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors.' % len(embeddings_index))

Total 400000 word vectors.


In [17]:
embedding_weights = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, index in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_weights[index] = embedding_vector

In [18]:
embedding_weights.shape

(90279, 100)

In [19]:
embedding_layer=Embedding(input_dim=len(word_index)+1,
                          output_dim=EMBEDDING_DIM,
                          weights=[embedding_weights],
                          input_length=MAX_SEQUENCE_LENGTH,
                          trainable=True)

In [20]:
#Input for each Question
left_question_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
right_question_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

#Encoders at Question Level
embedded_sequences = embedding_layer(left_question_input)
left_lstm_sent = LSTM(100,dropout_U=0.5,dropout_W=0.5)(embedded_sequences)

embedded_sequences = embedding_layer(right_question_input)
right_lstm_sent = LSTM(100,dropout_U=0.5,dropout_W=0.5)(embedded_sequences)

#merge two encoded inputs with the l1 distance between them
L1_distance = lambda x: K.abs(x[0]-x[1])
both = merge([left_lstm_sent,right_lstm_sent], mode = L1_distance, output_shape=lambda x: x[0])
prediction = Dense(2,activation='sigmoid')(both)

siamese_net = Model(input=[left_question_input,right_question_input],output=prediction)
siamese_net.compile(loss="binary_crossentropy",optimizer=Adam())

siamese_net.count_params()

print("model - Siamese Networks LSTM dropout:0.5")
print(siamese_net.summary())

model - Siamese Networks LSTM dropout:0.5
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 200)           0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 200)           0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 200, 100)      9027900     input_1[0][0]                    
                                                                   input_2[0][0]                    
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 100)     

In [21]:
#data_question2_1000_train.shape
#duplicates_1000_train=np.array(duplicates_1000_train).reshape(1000,1)
#duplicates_1000_vali=np.array(duplicates_1000_vali).reshape(1000,1)

In [22]:
# checkpoint
filepath="simple-lstm-dropout-5-5-glove-weights-improvement-{epoch:02d}-{val_loss:.3f}.hdf5"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint1]
# Fit the model
siamese_net.fit([questions1_sequences_train,questions2_sequences_train], duplicates_train,
                validation_data=([questions1_sequences_val,questions2_sequences_val],duplicates_val)
                                  ,nb_epoch=10, batch_size=50,callbacks=callbacks_list,verbose=0)

Epoch 00000: val_loss improved from inf to 0.51393, saving model to simple-lstm-dropout-5-5-glove-weights-improvement-00-0.514.hdf5
Epoch 00001: val_loss improved from 0.51393 to 0.46473, saving model to simple-lstm-dropout-5-5-glove-weights-improvement-01-0.465.hdf5
Epoch 00002: val_loss improved from 0.46473 to 0.45454, saving model to simple-lstm-dropout-5-5-glove-weights-improvement-02-0.455.hdf5
Epoch 00003: val_loss did not improve
Epoch 00004: val_loss did not improve
Epoch 00005: val_loss improved from 0.45454 to 0.44080, saving model to simple-lstm-dropout-5-5-glove-weights-improvement-05-0.441.hdf5
Epoch 00006: val_loss did not improve
Epoch 00007: val_loss did not improve
Epoch 00008: val_loss did not improve


KeyboardInterrupt: 